In [2]:
import os
import re
import json
import time
import librosa
import subprocess
import shlex
import torch
import fairseq
import numpy as np
from tqdm import tqdm


In [2]:
def get_lang(d):
    s = d.split('_')[0]
    s = re.sub(r'\d+$', '', s.split('-')[0]) if re.search('[a-zA-Z]+', s) else s
    return s

In [3]:
seen_utt = set()

In [8]:
# change split to 'train' / 'dev'

split = 'dev'
lang2data = {
    'BG': f'data/GlobalPhone/gp_Bulgarian_{split}/',
    'CZ': f'data/GlobalPhone/gp_Czech_{split}/',
    'PL': f'data/GlobalPhone/gp_Polish_{split}/',
}
metadata_path = f'dump/{split}/deltafalse/data.json'
with open(metadata_path, 'r') as f:
    data = json.load(f)
    data = data['utts']
all_langs = set(get_lang(k) for k in data.keys())

In [6]:
tempfile= 'tmp.wav'
sr = 16000
feat_folder = 'w2vfeat_npy' 
cp_path = '/mnt/ssd/espnet/egs/low-resource-language/asr1-slavic/wav2vec_small.pt'
model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([cp_path])
model = model[0]
model.eval()
device = torch.device('cuda:0')
model = model.to(device)

for name, paras in model.named_parameters():
    print(name)

mask_emb
feature_extractor.conv_layers.0.0.weight
feature_extractor.conv_layers.0.2.weight
feature_extractor.conv_layers.0.2.bias
feature_extractor.conv_layers.1.0.weight
feature_extractor.conv_layers.2.0.weight
feature_extractor.conv_layers.3.0.weight
feature_extractor.conv_layers.4.0.weight
feature_extractor.conv_layers.5.0.weight
feature_extractor.conv_layers.6.0.weight
post_extract_proj.weight
post_extract_proj.bias
quantizer.vars
quantizer.weight_proj.weight
quantizer.weight_proj.bias
project_q.weight
project_q.bias
encoder.pos_conv.0.bias
encoder.pos_conv.0.weight_g
encoder.pos_conv.0.weight_v
encoder.layers.0.self_attn.k_proj.weight
encoder.layers.0.self_attn.k_proj.bias
encoder.layers.0.self_attn.v_proj.weight
encoder.layers.0.self_attn.v_proj.bias
encoder.layers.0.self_attn.q_proj.weight
encoder.layers.0.self_attn.q_proj.bias
encoder.layers.0.self_attn.out_proj.weight
encoder.layers.0.self_attn.out_proj.bias
encoder.layers.0.self_attn_layer_norm.weight
encoder.layers.0.self_at

In [10]:
def line_count(filename):
    return int(subprocess.check_output(['wc', '-l', filename]).split()[0])

In [11]:

for lang in all_langs:
    data_dir = lang2data[lang]
    nline = line_count(f'{data_dir}/wav.scp')
    with open(f'{data_dir}/wav.scp', 'r') as f:
        for l in tqdm(f, total=nline):
            # BG051_UTT001 shorten -x /home/hertin/Workplace/SST/dataset/babel/S0319/Bulgarian/adc/051/BG051_1.adc.shn - | sox -t raw -r 16000 -b 16 -e signed-integer - -t wav - |
            uttid, cmd = l.strip().strip('- |').split(' ', maxsplit=1)
            cmd = f'{cmd} {tempfile}'
            
            process = subprocess.Popen(
                cmd,
                shell=True,
                stdout=subprocess.PIPE, 
                stderr=subprocess.PIPE
            )
            stdout, stderr = process.communicate()
            stdout, stderr
            audio, _ = librosa.load(tempfile, sr=sr)
    
            wav_input = torch.from_numpy(audio).to(device).unsqueeze(0)

            z = model.feature_extractor(wav_input)
            feat = z.squeeze(0).detach().cpu().numpy().T
            np.save(f'{feat_folder}/{uttid}.npy', feat)

            assert uttid not in seen_utt
            seen_utt.add(uttid)

100%|██████████| 830/830 [01:44<00:00,  7.91it/s]


In [49]:
f'{feat_folder}/{uttid}.npy'

'w2vfeat_npy/CZ009_UTT140.npy'

In [52]:
np.load(f'{feat_folder}/{uttid}.npy', mmap_mode='r').shape[1:]

(512, 649)

In [58]:
z.squeeze(0).T.shape

torch.Size([649, 512])

In [56]:
z.squeeze(0).shape


torch.Size([512, 649])

In [23]:
cp_path = '/home/hertin/Workplace/SST/low-resource/fairseq/examples/wav2vec/scripts/xlsr_53_56k.pt'
model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([cp_path])
model = model[0]
model.eval()
device = torch.device('cuda:0')
model = model.to(device)
# wav_input_16khz = torch.randn(2,10000)
start = time.time()
wav_input = torch.from_numpy(audio).to(device).unsqueeze(0)

z = model.feature_extractor(wav_input)
feat = z.detach().cpu().numpy()
end = time.time()
print(f'cost {end-start} s')
# c = model.feature_aggregator(z)

cost 0.011581659317016602 s


In [24]:
cp_path = '/home/hertin/Workplace/SST/low-resource/fairseq/examples/wav2vec/scripts/xlsr_53_56k.pt'
model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([cp_path])
model = model[0]
model.eval()

# wav_input_16khz = torch.randn(2,10000)
start = time.time()
wav_input = torch.from_numpy(audio).unsqueeze(0)

z = model.feature_extractor(wav_input)
end = time.time()
print(f'cost {end-start} s')
# c = model.feature_aggregator(z)

cost 0.9544930458068848 s


In [48]:
z.size()

torch.Size([1, 512, 649])